# Train Model on Chopin Nocturnes

In [1]:
import tensorflow as tf

from music_generator.model import MusicModel
from music_generator.serializers.discrete_time_serializer import DiscreteTimeMidiSerializer
import music_generator.utilities.sequence_utils as sequence_utils
import music_generator.utilities.utils as utils

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0-dev20200327


# Make sure CUDA and GPU are working

In [2]:
for message in utils.check_cuda_and_gpu():
    print(message)

CUDA and GPU Available.


# Create model

In [3]:
description = 'chopin_nocturnes'

model = MusicModel(
    ckpt_dir='./training_checkpoints/{}'.format(description),
    log_dir='./logs/{}'.format(description)
)
model.summary()

Model: "music_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         45568     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1312768   
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         

# Create Dataset

A training and validation dataset are created - the validation dataset is used in a callback to reduce the learning rate when it plateaus.

In [6]:
data_path = './training_data/chopin_nocturnes/'

serializer = DiscreteTimeMidiSerializer()
sequences = serializer.serialize_folder(data_path)
sequences, labels = sequence_utils.window(sequences)
dataset_train, dataset_val = sequence_utils.make_tf_datasets(sequences, labels)

Training Sequences: 123142
Validation Sequences: 6482


# Train Model on Dataset

In [7]:
epochs = 100
history = model.fit(dataset_train,
                    val_data=dataset_val,
                    epochs=epochs,
                    verbose=1)

TypeError: can only concatenate list (not "NoneType") to list

# Sample from model

In [ ]:
# Generate MIDI files using different seeds - these below represent a B-minor chord, a G7 chord, and an A-minor chord
seeds = [[38, 47, 54, 62, 66],
         [43, 55, 59, 62, 65],
         [45, 60, 64, 69]]

length = 1000

for seed in seeds:
    seed_string = '-'.join([str(x) for x in seed])
    generated_sequence = model.generate_sequence(length, seed)
    serializer.deserialize(generated_sequence, './generated_files/{}'.format(description), 'sample_{}.mid'.format(seed_string))